In [ ]:
import csv
import cv2 as cv

import numpy as np
import pandas as pd
import os

from glob import glob
import matplotlib.pyplot as plt
from keras import backend as K

from keras.applications import inception_v3, inception_resnet_v2, mobilenet, resnet50, vgg16
from keras.layers import dot, Dense, GlobalAveragePooling2D, Lambda, Input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.optimizers import RMSprop, SGD

## Import test image csv information

In [ ]:
data_dir = '/storage/data/humpback-whale-challenge'
test_dir = 'test'

test_dict = dict()

print("Test folder: %s" %(os.path.join(data_dir,test_dir)))

for i, fn in enumerate(glob(os.path.join(data_dir,test_dir,'*.jpg'))):
    # print(fn)
    test_dict[i] = fn

img = cv.imread(os.path.join(data_dir,test_dir,"326b70e9.jpg"))
img = np.array(img,dtype=np.float32)/255
# print(img)
plt.imshow(img,)

print("# of test images: %s" %str(len(test_dict)))

## Import whale database information from training set

In [ ]:
train_csv = 'train.csv'
train_dir = 'train'
df = pd.read_csv(filepath_or_buffer=os.path.join(data_dir,train_csv))

train_classes = df.groupby("Id").size()
print("\n# of unique database classes: %d" %train_classes.shape[0])

# create dictionary mapping labels to image filenames
db_img_filenames = df["Image"].tolist()
db_labels = df["Id"].tolist()
db = dict(zip(db_labels,db_img_filenames))

# remove new_whale key from dictionary
'''for i,key in enumerate(db.keys()):
    if key == "new_whale":
        print("New whale_idx: %d" %i)'''

del db["new_whale"]

# print(len(db))

# create dictionary mapping label index to label
labels_dict = {}
for i, label in enumerate(db.keys()):
    labels_dict[i]=label

# print(str(len(labels_dict)))

## Create database encodings

In [ ]:
# load image from disk
def load_img(fn):

    # print(os.path.join(data_dir,fn))

    img = cv.imread(filename=os.path.join(fn),flags=0)
    img = cv.resize(src=img, dsize=(224,224), dst=img)
    img = np.array(img, dtype=np.float32)
    img /=  255
    img = np.stack((img,img,img),axis=-1)

    return img

# load training images into database
def create_db_array():
    
    arr = []
    
    for db_fn in db.values():
        db_img = load_img(os.path.join(data_dir,train_dir,db_fn))
        arr.append(db_img)
    
    arr = np.array(arr, dtype=np.float32)    
    print(arr.shape)
        
    return arr
  
# choose pre-trained base network that generates a (n,1) feature vector for an input image
def create_pre_trained_base(input_shape, base_id):
    
    # Inception-v3
    if base_id is 0:
        base = inception_v3.InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False)
        base_name = 'Inception-V3'

    # MobileNet
    elif base_id is 1:
        base = mobilenet.MobileNet(input_shape=input_shape, weights='imagenet', include_top=False)
        base_name = 'MobileNet'

    # Inception-ResNet-v2
    elif base_id is 2:
        base = inception_resnet_v2.InceptionResNetV2(input_shape=input_shape, weights='imagenet', include_top=False)
        base_name = 'InceptionResNet-v2'

    # ResNet50
    elif base_id is 3:
        base = resnet50.ResNet50(input_shape=input_shape, weights='imagenet', include_top=False)
        base_name = 'ResNet50'

    # VGG16
    elif base_id is 4:
        base = vgg16.VGG16(input_shape=input_shape, weights='imagenet', include_top=False)
        base_name = 'VGG16'

    top = GlobalAveragePooling2D()(base.output)
    model = Model(inputs=base.input, outputs=top, name='base_model')
    
    print("\nBase Network: %s" % base_name)

    return model


db_arr = create_db_array()
  
base_id = 4  # 0 = Inception-v3, 1 = MobileNet, 2 = Inception-ResNet-v2, 3= ResNet50, 4=VGG16
img_shape = (224,224,3)
base_model = create_pre_trained_base(input_shape=img_shape, base_id=base_id)

db_encoding = base_model.predict(x=db_arr, batch_size=32, verbose=1)
print("\nDatabase Encoding: "+str(db_encoding.shape))
print("Database Size: "+str(db_encoding.nbytes/1e9))

## Create test encodings

In [ ]:
'''test_encodings = []

for i, test_img_fn in enumerate(test_dict.values()):

    test_img = load_img(test_img_fn)
    test_img = np.reshape(a=test_img, newshape=(1,224,224,3))
    encoding = base_model.predict(x=test_img, batch_size=1, verbose=0)
    
    test_encodings.append(encoding)

test_encodings = np.array(test_encodings,np.float32)
print("Test Encoding: "+str(test_encodings.shape))
print("Test Encoding Size: "+str(test_encodings.nbytes/1e9))'''
    

## Define Siamese Network

In [ ]:
weights_dir = 'weights'
weights_filename = 'crop_wts2.h5'

# calculate cosine distance b/t feature vector outputs from base network
def cos_distance(feat_vects):

    K.set_epsilon(1e-07)
    epsilon = K.epsilon()

    x1, x2 = feat_vects

    result = K.maximum(x=dot(inputs=[x1, x2], axes=1, normalize=True), y=epsilon)

    return result

# create a siamese model that calculates similarity b/t two feature vectors
def create_siamese_model(input_shape, enable_saved_wts=True):
    encoding_a = Input(shape=input_shape, name='input_a')
    encoding_b = Input(shape=input_shape, name='input_b')

    fc1_a = Dense(units=2048, activation='relu', kernel_regularizer=None, name='fc1_a')(encoding_a)
    fc1_b = Dense(units=2048, activation='relu', kernel_regularizer=None, name='fc1_b')(encoding_b)

    distance = Lambda(function=cos_distance, name='cos_distance', )([fc1_a, fc1_b])

    prediction = Dense(units=1, activation='sigmoid', kernel_regularizer=l2(l=0.0000), name='sigmoid')(distance)

    model = Model(inputs=[encoding_a, encoding_b], outputs=prediction, name='siamese_model')

    if enable_saved_wts is True:
        model.load_weights(filepath=weights_filename,by_name=True)
        print("Loaded weights: %s" %weights_filename)
        
    return model


encoding_shape = db_encoding.shape[1:]
siamese_model = create_siamese_model(encoding_shape)
siamese_model.compile(optimizer=RMSprop(),loss="binary_crossentropy")
print(siamese_model.summary())

## Test Evaluation

In [17]:
# return top-N database predictions
def find_topN(predictions, N):

    # find N indices with the highest similarity predictions
    idxs_topN = np.argsort(predictions)[-N:]

    labels = []

    # check if maximum prediction is <= 0.5. If so, assign "new_whale" class to top-1 prediction
    idx_max = idxs_topN[N-1]
    if predictions[idx_max] <= 0.5:
        labels.append("new_whale")
        stop_idx = 0
    else:
        # labels.append(labels_dict[idx_max])
        stop_idx = -1

    for i in range(N-1,stop_idx,-1):
        idx = int(idxs_topN[i])
        labels.append(labels_dict[idx])

    return labels

# load training images into database
def create_pairs(test_enc,db_encoding):
    
    arr = []
    # test_enc=np.reshape(a=test_enc,newshape=(512,))
    # print(test_enc.shape)
    
    for enc in db_encoding:
        arr.append([test_enc[0],enc])
        # print(enc.shape)
    
    # print(str(len(arr)))
    arr = np.array(arr,np.float32)    
    # print(arr.shape)
        
    return arr

# save top 5 results to csv file
def save_results(final_predictions):
    
    csv_fn = 'results.csv'

    with open(csv_fn, 'w') as f:
        writer = csv.writer(f, delimiter=',')

        for name, labels in final_predictions:
            l_concat = " ".join(labels)
            # print(l_concat)
            writer.writerow([name,l_concat])
    
    print("Saving top-5 predictions to %s" %csv_fn)

        
n_test = len(test_dict)
n_db = len(db)
results = np.zeros(shape=(n_test,n_db),dtype=np.float32)
final_predictions = []
N = 5

# iterate through all test image and db combinations
for i, test_img_fn in enumerate(test_dict.values()):
# for i, test_enc in enumerate(test_encodings):
    
    # if i > 10:
       # break
    
    test_img = load_img(test_img_fn)
    test_img = np.reshape(a=test_img, newshape=(1,224,224,3))
    test_enc = base_model.predict(x=test_img, batch_size=1, verbose=0)
    # print("test encoding shape: " +str(test_enc.shape))
    
    offset = len(test_dir)+len(data_dir)+2

    '''# calculate similarity b/t test image and all db images
    for j, db_enc in enumerate(db_encoding):          
        eval_pair = [[test_enc[0],db_enc]]
        eval_pair = np.array(eval_pair)
        # print(eval_pair.shape)
        prediction = siamese_model.predict(x=[eval_pair[:,0],eval_pair[:,1]], batch_size=1, verbose=0)
        results[i][j] = prediction[0]'''
    
    eval_pair = create_pairs(test_enc,db_encoding)
    # print(eval_pair.shape)
    prediction = siamese_model.predict(x=[eval_pair[:,0],eval_pair[:,1]], batch_size=32, verbose=1)
    results[i][0:n_db] = np.transpose(prediction)
    # print(100*results[i][0:10])
        
    # print(100*results[i][0:10])

    topN = find_topN(results[i],N)
    final_predictions.append([test_img_fn[offset:],topN])
    
    print("\nImage %d: %s" %(i,test_img_fn[offset:]))
    print("Top %d Predictions: %s" %(N,str(topN)))
    
    # save every 100 images
    if i%100==0:
        save_results(final_predictions)

save_results(final_predictions)

4250/4250 [==============================] - 0s 33us/step

Image 0: 1a5c10f5.jpg
Top 5 Predictions: ['w_ba56291', 'w_3136deb', 'w_983829b', 'w_36a853c', 'w_23e5a4c']
Saving top-5 predictions to results.csv
4250/4250 [==============================] - 0s 31us/step

Image 1: 6c681c2d.jpg
Top 5 Predictions: ['w_4f37147', 'w_8e93d0e', 'w_2173953', 'w_6193637', 'w_2dc2ef2']
4250/4250 [==============================] - 0s 33us/step

Image 2: 1758efcb.jpg
Top 5 Predictions: ['w_c87c10b', 'w_d1bbde9', 'w_7a3a328', 'w_06972d2', 'w_2957331']
4250/4250 [==============================] - 0s 34us/step

Image 3: 689b9fde.jpg
Top 5 Predictions: ['w_c786765', 'w_f792125', 'w_0a0cf7d', 'w_8a997b8', 'w_95af6a2']
4250/4250 [==============================] - 0s 33us/step

Image 4: b5a1f86d.jpg
Top 5 Predictions: ['w_b14007c', 'w_1746c88', 'w_c39c56e', 'w_164cfff', 'w_d76d3bb']
4250/4250 [==============================] - 0s 36us/step

Image 5: 81072584.jpg
Top 5 Predictions: ['w_aef3680', 'w_0ffc383', 'w_

4250/4250 [==============================] - 0s 30us/step

Image 49: 987cf047.jpg
Top 5 Predictions: ['w_8e4abc9', 'w_9c3db0a', 'w_9dc4181', 'w_ecbf9fa', 'w_bf38f05']
4250/4250 [==============================] - 0s 32us/step

Image 50: d0c34bcd.jpg
Top 5 Predictions: ['w_cda5d6e', 'w_572033c', 'w_2216a46', 'w_74257e8', 'w_37223f9']
4250/4250 [==============================] - 0s 38us/step

Image 51: 86a1b85f.jpg
Top 5 Predictions: ['w_1aa0526', 'w_ef7cdd5', 'w_25d7f93', 'w_7427ace', 'w_731e3aa']
4250/4250 [==============================] - 0s 32us/step

Image 52: 7a371004.jpg
Top 5 Predictions: ['w_0b7e949', 'w_dd5df89', 'w_4848a3c', 'w_25fdcfb', 'w_7419e4b']
4250/4250 [==============================] - 0s 32us/step

Image 53: ff4ed0f6.jpg
Top 5 Predictions: ['w_2d57dce', 'w_ba0366f', 'w_ed0c4cc', 'w_8a00b09', 'w_c6439b8']
4250/4250 [==============================] - 0s 33us/step

Image 54: 738ec4a1.jpg
Top 5 Predictions: ['w_18fbec1', 'w_679f2eb', 'w_989fbbb', 'w_e0e5c9e', 'w_985877e'

4250/4250 [==============================] - 0s 32us/step

Image 98: c0fa1c25.jpg
Top 5 Predictions: ['w_9f27ca9', 'w_b34793e', 'w_373c114', 'w_7a95674', 'w_80211fd']
4250/4250 [==============================] - 0s 34us/step

Image 99: bc9b6529.jpg
Top 5 Predictions: ['w_efd3f81', 'w_4d358e5', 'w_e613f12', 'w_1d53d9c', 'w_3d3c0f9']
4250/4250 [==============================] - 0s 32us/step

Image 100: ece33956.jpg
Top 5 Predictions: ['w_97eaa0e', 'w_cd3e1bd', 'w_e964ae8', 'w_03c6d18', 'w_9b3ed01']
Saving top-5 predictions to results.csv
4250/4250 [==============================] - 0s 33us/step

Image 101: 3be626ef.jpg
Top 5 Predictions: ['w_5bc7e7f', 'w_126653d', 'w_e09d8cf', 'w_fdc8bf6', 'w_06c470d']
4250/4250 [==============================] - 0s 31us/step

Image 102: dad24756.jpg
Top 5 Predictions: ['w_17b33ae', 'w_c206494', 'w_78a4639', 'w_46b211d', 'w_47f29cd']
4250/4250 [==============================] - 0s 32us/step

Image 103: 5d97b3bf.jpg
Top 5 Predictions: ['w_2386a5a', 'w_d7d

4250/4250 [==============================] - 0s 33us/step

Image 147: 74463545.jpg
Top 5 Predictions: ['w_49ae15d', 'w_7cd711c', 'w_0526c04', 'w_bd5cf0a', 'w_931ade2']
4250/4250 [==============================] - 0s 33us/step

Image 148: 44023ae4.jpg
Top 5 Predictions: ['w_733fe81', 'w_dd60c18', 'w_d25a717', 'w_8c957e0', 'w_2bea4c4']
4250/4250 [==============================] - 0s 32us/step

Image 149: 35df51b7.jpg
Top 5 Predictions: ['w_b05f565', 'w_0f96780', 'w_b80d2cb', 'w_17dc953', 'w_bf69a19']
4250/4250 [==============================] - 0s 33us/step

Image 150: a8020970.jpg
Top 5 Predictions: ['w_759b647', 'w_faa4239', 'w_8166c79', 'w_62ca7c8', 'w_b4d182b']
4250/4250 [==============================] - 0s 32us/step

Image 151: a349d259.jpg
Top 5 Predictions: ['w_85f9040', 'w_c1b3568', 'w_230a0de', 'w_8d3b8b6', 'w_e7a238b']
4250/4250 [==============================] - 0s 32us/step

Image 152: b4f5e498.jpg
Top 5 Predictions: ['w_d3857a6', 'w_fe054f3', 'w_c87651d', 'w_01cbcbf', 'w_c3

4250/4250 [==============================] - 0s 33us/step

Image 196: 486f6880.jpg
Top 5 Predictions: ['w_e4616da', 'w_ea90267', 'w_43966a3', 'w_0970c7f', 'w_30d8376']
4250/4250 [==============================] - 0s 32us/step

Image 197: bb1ae392.jpg
Top 5 Predictions: ['w_f801078', 'w_5f2b407', 'w_8f28757', 'w_3694c7d', 'w_9d48bcf']
4250/4250 [==============================] - 0s 31us/step

Image 198: 2d83ecf4.jpg
Top 5 Predictions: ['w_aaafb64', 'w_b5ea8a0', 'w_78948b5', 'w_0f96780', 'w_17dc953']
4250/4250 [==============================] - 0s 32us/step

Image 199: 575d53aa.jpg
Top 5 Predictions: ['w_39d2684', 'w_e4e6f5b', 'w_36a853c', 'w_a34c992', 'w_0f92544']
4250/4250 [==============================] - 0s 37us/step

Image 200: 53cdca34.jpg
Top 5 Predictions: ['w_eab49b7', 'w_6257bdf', 'w_e1f55ad', 'w_9f1fafb', 'w_dd52cfc']
Saving top-5 predictions to results.csv
4250/4250 [==============================] - 0s 33us/step

Image 201: bc489745.jpg
Top 5 Predictions: ['w_1a70685', 'w_0

...

4250/4250 [==============================] - 0s 32us/step

Image 15382: 03df3506.jpg
Top 5 Predictions: ['w_4659acf', 'w_c02ac49', 'w_ad32646', 'w_a700fc9', 'w_87050a3']
4250/4250 [==============================] - 0s 32us/step

Image 15383: 42e7d8e6.jpg
Top 5 Predictions: ['w_6ddbb81', 'w_8c2de31', 'w_816d462', 'w_41ec10a', 'w_e7b1cb3']
4250/4250 [==============================] - 0s 33us/step

Image 15384: cf0f9e55.jpg
Top 5 Predictions: ['w_5292ff4', 'w_3a9ee71', 'w_de2c79f', 'w_f544197', 'w_2ce5ce2']
4250/4250 [==============================] - 0s 35us/step

Image 15385: 45b9f00f.jpg
Top 5 Predictions: ['w_b70a0c4', 'w_5010531', 'w_e6ced04', 'w_6a4e5f2', 'w_6545984']
4250/4250 [==============================] - 0s 31us/step

Image 15386: de098549.jpg
Top 5 Predictions: ['w_dd60c18', 'w_64d8a6d', 'w_0da6f67', 'w_29fc831', 'w_07616fd']
4250/4250 [==============================] - 0s 30us/step

Image 15387: a52b0a4c.jpg
Top 5 Predictions: ['w_35eb420', 'w_4f6c7e0', 'w_af06773', 'w_6d0

4250/4250 [==============================] - 0s 31us/step

Image 15430: 8bb14253.jpg
Top 5 Predictions: ['w_ce7c6c0', 'w_47c5b29', 'w_5982299', 'w_40be51f', 'w_ca56bf3']
4250/4250 [==============================] - 0s 32us/step

Image 15431: 75c8c6d5.jpg
Top 5 Predictions: ['w_8e93d0e', 'w_8f28757', 'w_e58fbe3', 'w_2173953', 'w_4655b25']
4250/4250 [==============================] - 0s 32us/step

Image 15432: 1b587e8b.jpg
Top 5 Predictions: ['w_059df09', 'w_aef3680', 'w_7c943ab', 'w_46a4473', 'w_5dfafa5']
4250/4250 [==============================] - 0s 32us/step

Image 15433: 0915ea70.jpg
Top 5 Predictions: ['w_ce425e5', 'w_269b090', 'w_aa7e5f4', 'w_96fd936', 'w_b067417']
4250/4250 [==============================] - 0s 34us/step

Image 15434: 31a81c2e.jpg
Top 5 Predictions: ['w_680b86e', 'w_6f40f26', 'w_616ca36', 'w_018bc64', 'w_3a7d86d']
4250/4250 [==============================] - 0s 33us/step

Image 15435: dc649b21.jpg
Top 5 Predictions: ['w_7292419', 'w_f8024be', 'w_cae7677', 'w_46b

4250/4250 [==============================] - 0s 32us/step

Image 15478: 650f3b6f.jpg
Top 5 Predictions: ['w_29f00ae', 'w_7cfaa82', 'w_df82632', 'w_3aa2073', 'w_3d3c0f9']
4250/4250 [==============================] - 0s 32us/step

Image 15479: fc09cf94.jpg
Top 5 Predictions: ['w_1e4e7a4', 'w_23dce10', 'w_a14d6d8', 'w_0a0cf7d', 'w_6918155']
4250/4250 [==============================] - 0s 32us/step

Image 15480: 06b0cf32.jpg
Top 5 Predictions: ['w_fe6f08d', 'w_03a2ed7', 'w_a3885dc', 'w_642de28', 'w_bdbec6c']
4250/4250 [==============================] - 0s 32us/step

Image 15481: d60fdb4b.jpg
Top 5 Predictions: ['w_c6fceea', 'w_881f3c4', 'w_02d7dc8', 'w_b170173', 'w_94e071d']
4250/4250 [==============================] - 0s 32us/step

Image 15482: dd39b81f.jpg
Top 5 Predictions: ['w_710a3a3', 'w_c87c10b', 'w_c352046', 'w_733fe81', 'w_facffea']
4250/4250 [==============================] - 0s 31us/step

Image 15483: cb90195c.jpg
Top 5 Predictions: ['w_871b1b9', 'w_33e7def', 'w_777f59a', 'w_8ec

4250/4250 [==============================] - 0s 33us/step

Image 15526: 70bea21a.jpg
Top 5 Predictions: ['w_ba56291', 'w_9c3db0a', 'w_c13b4e6', 'w_983829b', 'w_aff0877']
4250/4250 [==============================] - 0s 32us/step

Image 15527: 62070da6.jpg
Top 5 Predictions: ['w_db93738', 'w_0bc1db0', 'w_e09d8cf', 'w_a29719a', 'w_12d9132']
4250/4250 [==============================] - 0s 33us/step

Image 15528: 43f85bb5.jpg
Top 5 Predictions: ['w_aef3680', 'w_7142b19', 'w_6315bc6', 'w_4ce0510', 'w_7c943ab']
4250/4250 [==============================] - 0s 33us/step

Image 15529: e1b2de44.jpg
Top 5 Predictions: ['w_ad6edbe', 'w_8ae97a4', 'w_fd698a3', 'w_64befc3', 'w_2f6ad07']
4250/4250 [==============================] - 0s 31us/step

Image 15530: 97d67955.jpg
Top 5 Predictions: ['w_b05f565', 'w_9dc4181', 'w_cf7eebd', 'w_b80d2cb', 'w_8697f5f']
4250/4250 [==============================] - 0s 32us/step

Image 15531: 758385cc.jpg
Top 5 Predictions: ['w_d428980', 'w_0ebd514', 'w_0f96780', 'w_4a8

4250/4250 [==============================] - 0s 33us/step

Image 15574: a1ed01c0.jpg
Top 5 Predictions: ['w_b001dbe', 'w_c6be61e', 'w_c87651d', 'w_5e43abb', 'w_c26666e']
4250/4250 [==============================] - 0s 40us/step

Image 15575: dfc9b491.jpg
Top 5 Predictions: ['w_14964c1', 'w_7c943ab', 'w_861cc1c', 'w_6375873', 'w_9b00e07']
4250/4250 [==============================] - 0s 32us/step

Image 15576: f919315b.jpg
Top 5 Predictions: ['w_c95a0ea', 'w_3aa2073', 'w_dd0c2e0', 'w_0c93f94', 'w_18df014']
4250/4250 [==============================] - 0s 32us/step

Image 15577: 27ba48fc.jpg
Top 5 Predictions: ['w_f8cf8ca', 'w_552ec4e', 'w_6865ccb', 'w_46540b1', 'w_ce01917']
4250/4250 [==============================] - 0s 30us/step

Image 15578: f9b61ffa.jpg
Top 5 Predictions: ['w_01a4234', 'w_7bdfc11', 'w_10ace22', 'w_42ab867', 'w_314d77b']
4250/4250 [==============================] - 0s 31us/step

Image 15579: ff7fee1d.jpg
Top 5 Predictions: ['w_680b86e', 'w_018bc64', 'w_b478c13', 'w_7e9

## Save top-5 predictions to csv

In [ ]:
# save_dir = 'results'

